In [ ]:
# Check first whether GPU is on

0. Prepare before training

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# path alias
!ln -s /content/gdrive/MyDrive/Colab_Notebooks /mydrive

In [ ]:
# check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
# Check CUDA version to bring the right cudnn file
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# prepare the proper cudnn file in your google drive, and unzip it
!tar -xzvf "/content/gdrive/MyDrive/Colab_Notebooks/yolov3/cudnn-11.3-linux-x64-v8.2.0.53.tgz" -C /usr/local

In [ ]:
# Check whether unzip was completed well
!cat /usr/local/cuda/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

1. Clone, configure & compile Darknet

In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Compile darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

2. Copy & customize configuration file

In [ ]:
# Make a copy of yolov4.cfg
!cp cfg/yolov4.cfg cfg/yolov4_training.cfg

In [ ]:
# Change lines in yolov4_training.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov4_training.cfg
!sed -i 's/subdivisions=8/subdivisions=64/' cfg/yolov4_training.cfg
!sed -i 's/max_batches = 500500/max_batches = 6000/' cfg/yolov4_training.cfg
!sed -i 's/steps=400000,450000/steps = 4800,5400/' cfg/yolov4_training.cfg
!sed -i '968 s@classes=80@classes=3@' cfg/yolov4_training.cfg
!sed -i '1056 s@classes=80@classes=3@' cfg/yolov4_training.cfg
!sed -i '1144 s@classes=80@classes=3@' cfg/yolov4_training.cfg
!sed -i '961 s@filters=255@filters=24@' cfg/yolov4_training.cfg
!sed -i '1049 s@filters=255@filters=24@' cfg/yolov4_training.cfg
!sed -i '1137 s@filters=255@filters=24@' cfg/yolov4_training.cfg

3. Create ".names" and ".data" files

In [ ]:
!echo -e 'pedestrian\nperson on bike\nbike' > data/obj.names

# Must create the directory for backup before executing the code below
!echo -e 'classes= 3\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov4/main/backup' > data/obj.data

4. Save "yolov4_training.cfg" and "obj.names" files in Goodgle Drive

In [ ]:
!cp cfg/yolov4_training.cfg /mydrive/yolov4/main/yolov4_trial3_training_final.cfg
!cp data/obj.names /mydrive/yolov4/main/yolov4_trial3_classes.txt

5. Create a folder and unzip image dataset

In [ ]:
!mkdir data/obj

# use images in 'yolov3' folder as train data is same
!unzip /mydrive/yolov3/images.zip -d data/obj

6. Create train.txt file

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

7. Download pre-trained weights for the convolutional layers file

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

8. Train!

In [ ]:
# Initial execution : Transfer Learning with yolov4.conv.137
!LD_LIBRARY_PATH=/usr/local/cuda/lib64/ ./darknet detector train data/obj.data cfg/yolov4_training.cfg yolov4.conv.137 -dont_show

# Uncomment the code below, if the Google colab connection was out and you want to continue training from where the previous training was ended
# !LD_LIBRARY_PATH=/usr/local/cuda/lib64/ ./darknet detector train data/obj.data cfg/yolov4_training.cfg /mydrive/yolov4/main/backup/yolov4_testing_last.weights -dont_show